In [1]:

from numpy import asarray
from PIL import Image
import numpy as np


def fancy_pca(image_path):
    #转化为numpy array
    im = Image.open(image_path) #load image.jpg
    i_a = asarray(im) #convert image to array
    #归一化
    i_a = i_a / 255.0
    #reshape成RGB三个维度
    img_rs = i_a.reshape(-1, 3)
    #中心化
    img_centered = img_rs - np.mean(img_rs, axis=0)
    #求协方差矩阵
    img_cov = np.cov(img_centered, rowvar=False)
    print(img_cov)
    #特征值分解
    eig_vals, eig_vecs = np.linalg.eigh(img_cov)
    print(eig_vals)
    print(eig_vecs)
    #特征值与特征向量排序
    sort_perm = eig_vals[::-1].argsort()
    eig_vals[::-1].sort()
    eig_vecs = eig_vecs[:, sort_perm]
    #特征值按列保存（?）
    m1 = np.column_stack((eig_vecs))
    print(m1)
    #采样alpha，计算增量
    m2 = np.zeros((3, 1))
    alpha = np.random.normal(0, 0.1)
    m2[:, 0] = alpha * eig_vals[:]
    add_vect = np.matrix(m1) * np.matrix(m2)
    #每一个像素值加上增量
    for idx in range(3):   # RGB
        i_a[..., idx] += add_vect[idx]
    #转换为0-255的范围
    orig_img = np.clip(i_a, 0.0, 255.0)
    orig_img = orig_img.astype(np.uint8)
    #转化为PIL的Image格式
    final_img = Image.fromarray(orig_img)

    return final_img

In [ ]:
"""
fancy PCA在一个batch上的实现
"""
# Import required libraries
from numpy import asarray
import argparse
import fancy_pca
from PIL import Image
import os
import glob
import numpy as np


# Load multiple images
image_list = []
fname_list = []
path = '/content/drive/My Drive/images/orig/lori.jpg' # path of the original image dataset. 
path2 = '/content/drive/My Drive/images/orig/' # path of the original dataset folder

# Load images and put them in a list (image_list)
for infile in glob.glob(path):
    im = Image.open(infile)
    image_list.append(im)

# Extract the names of the files and put them in a list (fname_list)
base = os.listdir(path2)
for f in base:
    fname = os.path.splitext(f)
    fname_list.append(fname[0])
    print("Loading",infile)
    print(len(image_list)," images has loaded.")


# Convert images to numpy arrays
array_list =[]
n=1
for i in image_list:
    i_a = asarray(i)
    array_list.append(i_a)
    print("Image", n, ": Conversion successful.")
    n+=1
print("Array of original image: ", i_a) #To see the array

print("Conversion successful")
print(type(i_a), i_a.shape)

# Perform the PCA color augmentation
n=1
aug_list=[]
for a in array_list:
    augmented = fancy_pca.fancy_pca(a)
    aug_list.append(a)
    print("Array",n, ":Augmentation successful")
    n += 1
print("Array of PCA augmented image: ", augmented) #To see the array

# Convert Fancy PCA result back to PIL image
path3 = "/content/drive/My Drive/images/aug/" #path of the destination folder
idx=0
for aug in aug_list:
    i2 = Image.fromarray(aug)
    print(fname_list)
    while idx<= len(fname_list):
        print(str(fname_list[idx])+"_1.jpg")
        i2.save(path3+(str(fname_list[idx])+"_1.jpg"))
        break
    idx+=1
    print("Augmented image", idx, "saved.")